In [1]:
import rs_classes.async_request_client as rs
import rs_classes.hooks as hs
import rs_functions.fetch_annotations_meta as fetch_annotations_meta
import rs_functions.fetch_annotation_content as fetch_annotation_content
import rs_functions.mongo_viewer_functions as mvf

token = ""
base_url = ""

client = rs.AsyncRequestClient(token, base_url)

In [ ]:
annotation_list = []

annotations_collection = await fetch_annotations_meta.get_annotation_meta(client, annotation_list)
await fetch_annotation_content.get_annotation_content(client, annotations_collection)
hooks = await mvf.collect_hooks_per_annotation(client, annotations_collection)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import json

target_schema_id = "vendor_match"

for annotation_id, annotation in annotations_collection.items():
    print(f"\033[33m Analysing annotation: {annotation_id}.. \033[0m")      
    for hook in annotation.related_hooks:        
        hook_obj =hooks.get_hook(hook.split("/")[-1])
        if hook_obj.hook_template == "https://us.app.rossum.ai/api/v1/hook_templates/39": 
            configuration_number = 0                                   
            for configuration in hook_obj.settings.get("configurations", None):                                
                if target_schema_id == configuration["mapping"]["target_schema_id"]:
                    configuration_number += 1                                        
                    print( f"\033[36m Hook {hook_obj.id} - {hook_obj.name} has configuration #:{configuration_number} with target_schema_id = {target_schema_id}\033[0m")                                                                                                    
                    for query in configuration["source"]["queries"]:                                                                        
                        mvf.find_and_replace_placeholder(query, annotation.annotation_content)                        
                        if query.get("find"):                                                        
                            result = await client.data_storage_find(collectionName=configuration["source"]["dataset"], query=query["find"])                            
                        elif query.get("aggregate"):
                            result = await client.data_storage_aggregate(collectionName=configuration["source"]["dataset"], pipeline=query["aggregate"])                                                        
                        else:
                            print("No Find or Aggregate has been found")
                        
                        # Formatting query and result for display
                        query_display = widgets.Textarea(
                            value=json.dumps(query, indent=4),
                            description='Query:',
                            layout=widgets.Layout(width='100%', height='250px'),
                            style={'description_width': 'initial'},
                        )
                        result_display = widgets.Textarea(
                            value=json.dumps(result["result"], indent=4) if result else "No result",
                            description='Result:',
                            layout=widgets.Layout(width='100%', height='250px')
                        )
                        display(widgets.VBox([query_display, result_display]))                         